In [1]:
from openai import OpenAI

In [2]:
from google import genai

In [3]:
from dotenv import load_dotenv

In [4]:
import pandas as pd

In [5]:
import toml

In [6]:
from rich.console import Console
from rich.markdown import Markdown

In [7]:
from pydantic import BaseModel

In [8]:
CONSOLE = Console()

In [9]:
def print(text):
    """text: str"""
    CONSOLE.print(Markdown(text))

In [10]:
def print_questions(user_qs):
    """user_qs: UserQueries"""
    for qs in user_qs.queries:
        print(qs)

In [11]:
parsed_toml = toml.load('../.streamlit/secrets.toml')

In [12]:
openai_key = parsed_toml['OPENAI_API_KEY']

In [13]:
gclient = genai.Client(api_key=parsed_toml['GEMINI_API_KEY'])

In [14]:
resposne = gclient.models.generate_content(
    model="gemini-2.5-flash",
    contents="Give me a list of user personas who would be using Fiddler for AI/ML Monitoring and observability"
)

In [15]:
print(resposne.text)

Fiddler AI is a powerful platform for monitoring, explaining, and analyzing AI models in production. As such, the  
user personas for Fiddler often span technical and non-technical roles, each with unique needs related to AI/ML    
monitoring and observability.                                                                                      

Here's a list of user personas who would be using Fiddler for AI/ML Monitoring and observability:                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                               1. The ML Ops Engineer (The "Guardian of Production")                               

 • Role/Title: ML Ops Engineer, Production ML Engineer, Senior DevOps Engineer (with ML focus)                     
 • Primary Goal: Ensure the continuous health, performance, and reliability of deployed AI models in production.   
   Minimize downtime and quickly diagnose issues.                                                                  
 • Key Pain Points: Unexplained model performance drops, silent failures, alert fatigue, difficulty pinpointing    
   root causes of issues (data vs. model), lack of visibility into model serving infrastructure.                   
 • How they use Fiddler:                                                                                           
    • Set up and monitor real-time alerts for performance degradation, data drift, and concept drift.              
    • Dashboard monitoring for overall model health, latency, and throughput.                                      
    • Drill down into specific model issues identified by alerts.                                                  
    • Integrate Fiddler alerts with existing ops dashboards (e.g., PagerDuty, Slack).                              
    • Track resource utilization and serving metrics related to model inference.                                   
 • Metrics they care about: Model throughput, latency, error rates, data ingress/egress, alert frequency, overall  
   system uptime, specific model performance metrics (e.g., F1-score, accuracy) after an alert.                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                   2. The Data Scientist (The "Model Whisperer")                                   

 • Role/Title: Data Scientist, Senior Data Scientist, ML Scientist                                                 
 • Primary Goal: Understand why their models are performing as they are in the real world, identify the need for   
   retraining, and debug model behavior.                                                                           
 • Key Pain Points: Models decaying over time (drift), unexpected model predictions, difficulty explaining model   
   decisions to stakeholders, needing to manually re-evaluate models in production, blind spots once models are    
   deployed.                                                                                                       
 • How they use Fiddler:                                                                                           
    • Deep dive into data drift and concept drift reports to understand why model performance is changing.         
    • Utilize explainability features (e.g., SHAP, LIME) to understand individual predictions or global model      
      behavior.                                                                                                    
    • Compare model performance against baselines and retrain triggers.                                            
    • Debug specific model prediction failures by inspecting input features and explanations.                      
    • Generate reports for model validation and re-training justification.                                         
 • Metrics they care about: Model accuracy, precisi

In [16]:
class UserQueries(BaseModel):
    queries: list[str]

In [17]:
QS_TEMPLATE = """Give me a list of {num_questions} queries that a user would want to ask FiddleBot about their models and projects in Fiddler.

Fiddler has the following capabilities:
- Monitor performance of ML models and generative AI applications
- Protect your LLM and GenAI applications with Guardrails
- Analyze model behavior to identify issues and opportunities
- Improve AI systems through actionable insights

Implementing Fiddler ML/LLM monitoring requires just three steps:
1. Onboard your ML/LLM application to Fiddler by defining its inputs, outputs, and related metadata
2. Publish your application data to Fiddler, typically the "digital exhaust" from your model serving platform
3. Monitor performance through dashboards and alerts that track the metrics most important to your use case

Fiddler automatically handles the complex work of generating metrics, detecting anomalies, and providing the visualizations you need to maintain high-quality ML applications.

Fiddlebot aims to be a chat based UI to the Fiddler platform, where users can ask the bot to fetch information for them. It is limited to the preprod environment on Fiddler, so please keep that in mind.
This is also an unoptimized version and the aim of having this internal dry-run is to ascertain where and how the agent is failing, along with usage patterns. This will better inform us when it comes to optimizing performance.
There are 3 major components working under the hood.
- Chatbot : Chat UI
- Plan n Solve : Certain queries require multiple steps to be resolved. The PnS module aims to first generate a plan and solve each step.
- MCP Server : Model Context Protocol Server. A set of tools that the agent can utilise in solving for the query. Runs the Fiddler python client under the hood.

Please keep in mind that the bot is still a work in progress and failure is expected. The aim of this exercise is to catch how the agent fails when it is faced with ambiguous queries, queries with incomplete parameters (ex asking for model schema without providing project) and queries which are beyond the scope of the agent.

FiddleBot has access the following capabilities via tools:
- list all projects in fiddler
- list all models in a project
- get model schema
- get model specs
- list alert rules for a model
- list triggered alerts for a rule
- list all custom metrics for a model

{condition}
"""

In [18]:
AMBIGUOUS_CONDITION = """The user is new to Fiddler and is not aware of its capabilities and what to ask for.
Their questions are not specific and ambiguous and incomplete.
"""

In [19]:
CONFLICTING_CONDITION = """The user wants to test the Fiddler expert and asks questions with conflicting information and requirements"""

In [20]:
GOOD_CONDITION = """The user is familiar with Fiddler and is aware of its capabilities and what to ask for.
The questions that the user asks are specific and clear. There is no ambiguity in the questions.
"""

In [21]:
response = gclient.models.generate_content(
    model="gemini-2.5-flash",
    contents=QS_TEMPLATE.format(num_questions=40, condition=AMBIGUOUS_CONDITION),
    config={
        "response_mime_type": "application/json",
        "response_schema": UserQueries
    }
)

In [22]:
queries = response.parsed

In [23]:
print_questions(response.parsed)

What's going on in Fiddler?

Tell me about my models.

Are there any active projects?

Show me some models.

What kind of data do you have for a model?

...

Are there any recommended actions?

What kind of guardrails are in place?

Show me a summary.

What's the most important information?

Can you tell me about the model called 'production_model'?

In [24]:
good_response = gclient.models.generate_content(
    model="gemini-2.5-flash",
    contents=QS_TEMPLATE.format(num_questions=40, condition=GOOD_CONDITION),
    config={"response_mime_type": "application/json", "response_schema": UserQueries},
)
good_qs = response.parsed
print_questions(good_qs)

What's going on in Fiddler?

Tell me about my models.

Are there any active projects?

Show me some models.

What kind of data do you have for a model?

...

Are there any recommended actions?

What kind of guardrails are in place?

Show me a summary.

What's the most important information?

Can you tell me about the model called 'production_model'?

In [25]:
good_response = gclient.models.generate_content(
    model="gemini-2.5-flash",
    contents=QS_TEMPLATE.format(num_questions=40, condition=GOOD_CONDITION),
    config={"response_mime_type": "application/json", "response_schema": UserQueries},
)
good_qs = response.parsed
print_questions(good_qs)

What's going on in Fiddler?

Tell me about my models.

Are there any active projects?

Show me some models.

What kind of data do you have for a model?

...

Are there any recommended actions?

What kind of guardrails are in place?

Show me a summary.

What's the most important information?

Can you tell me about the model called 'production_model'?

In [26]:
import fiddler as fdl

In [27]:
import traceback

In [28]:
BASE_URL = "https://preprod.cloud.fiddler.ai"
TOKEN = parsed_toml["FIDDLER_ACCESS_TOKEN"]
fdl.init(url=BASE_URL, token=TOKEN)

250626T07:44:24.707Z     INFO| attached stderr handler to logger: auto_attach_log_handler=True, and root logger not configured 
250626T07:44:24.709Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/server-info GET -- emit req (0 B, timeout: (5, 15)) 
250626T07:44:25.678Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/server-info GET -- resp code: 200, took 0.968 s, resp/req body size: (912 B, 0 B) 
250626T07:44:25.681Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/version-compatibility GET -- emit req (0 B, timeout: (5, 15)) 
250626T07:44:26.013Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/version-compatibility GET -- resp code: 200, took 0.330 s, resp/req body size: (2 B, 0 B) 


In [46]:
def _list_all_projects() -> list[str]:
    """
    List the names of all projects in the organisation
    """
    print("Listing all projects")
    project_names = []
    projects = list(fdl.Project.list())
    try:
        for project in projects:
            project_names.append(str(project.name))
        return project_names
    except Exception as e:
        return "Error in obtaining projects"

In [30]:
# @server.tool()
def _list_models_in_project(project_name: str) -> list[str]:
    """
    List out all model names associated with a project

    Args:
        project_name: Name of the project
    """
    try:
        project = fdl.Project.from_name(name=project_name)
        model_names = []
        for model in project.models:
            model_names.append(str(model.name))
    except Exception as e:
        return "Error in obtaining models"
    return model_names

In [31]:
def list_all_models_in_all_projects_in_organisation() -> list[str]:
    """
    Tool to list all model names across all projects in the organisation.
    """
    try:
        print("Listing all projects")
        model_names = []
        project_names = _list_all_projects()
        if not isinstance(project_names, list):
            return "Error in obtaining projects"
        print(f"Found {len(project_names)} projects")
        for project_name in project_names:
            print(f"Listing models in project: {project_name}")
            model_list = _list_models_in_project(project_name)
            if isinstance(model_list, list):
                model_names.extend(model_list)
            else:
                continue
        return model_names
    except Exception as e:
        print(f"Error in obtaining models: {traceback.format_exc()}")
        return "Error in obtaining models"

In [39]:
project_names = _list_all_projects()

Listing all projects

250626T07:46:41.191Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:46:41.745Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.554 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:46:41.749Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:46:42.101Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.352 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:46:42.106Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:46:42.403Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.296 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:46:42.409Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:46:42.717Z     INFO| http:

In [40]:
project_names

['jenn_llama_classifier']

In [41]:
projects = list(fdl.Project.list())

250626T07:47:00.233Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:47:00.809Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.575 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:47:00.814Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:47:01.108Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.294 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:47:01.112Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:47:01.411Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.298 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:47:01.416Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:47:01.717Z     INFO| http:

In [42]:
len(projects)

212

In [45]:
projects[1].name

'wb_chart_deletion_yooo'

In [ ]:
names = []
for project in projects:
    names.append(str(project.name))

In [47]:
all_models = list_all_models_in_all_projects_in_organisation()

Listing all projects

Listing all projects

250626T07:52:06.496Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:52:08.011Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 1.515 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:52:08.018Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:52:08.313Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.294 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:52:08.318Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:52:08.609Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.290 s, resp/req body size: (0.014 MB, 0 B) 
250626T07:52:08.615Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:52:09.248Z     INFO| http:

Found 212 projects

Listing models in project: jenn_llama_classifier

...

250626T07:54:48.303Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:54:48.581Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.278 s, resp/req body size: (449 B, 0 B) 
250626T07:54:48.582Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:54:49.061Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- resp code: 200, took 0.478 s, resp/req body size: (965 B, 0 B) 


Listing models in project: quickstart_examplesxxxxxxxxxxxxx

250626T07:54:49.066Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:54:49.340Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.273 s, resp/req body size: (459 B, 0 B) 
250626T07:54:49.342Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:54:49.742Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- resp code: 200, took 0.399 s, resp/req body size: (159 B, 0 B) 


Listing models in project: quickstart_examples_sreis

250626T07:54:49.747Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:54:50.026Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/projects GET -- resp code: 200, took 0.277 s, resp/req body size: (452 B, 0 B) 
250626T07:54:50.027Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- emit req (0 B, timeout: (5, 100)) 
250626T07:54:50.487Z     INFO| http: https://preprod.cloud.fiddler.ai/v3/models GET -- resp code: 200, took 0.459 s, resp/req body size: (988 B, 0 B) 


In [48]:
len(all_models)

228

In [53]:
raw_res = [
    "llama_classifier",
    "llama_classifier_temp_1",
    "bank_churn_simple_monitoring",
    "fiddler_llama_as_a_judge",
    "fiddler_llama_as_a_judge",
    "model",
    "model",
    "model",
    "model",
    "model",
    "model",
    "fiddler_llama_as_a_judge",
    "fiddler_llama_as_a_judge",
    "rag_chatbot",
    "model",
    "my_model",
    "yolo_v5su",
    "customer_chatbot",
    "resnet18",
    "resnet18",
    "fe5_low_threshold",
    "my_model",
    "huntington_ctr_model",
    "churn_classifier",
    "auto_insurance_binary_class",
    "fiddler_rag_llm_chatbot_njn1",
    "bank_churn_simple_njn2",
    "churn_classifier",
    "model",
    "model",
    "nlp_newsgroups_multiclass",
    "airline_delay_model",
    "bank_churn_simple_monitoring",
    "diabetic_multiclass",
    "diabetic_multiclass1",
    "fiddler_llm_evaluation",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbotp",
    "search_ranking_example",
    "search_ranking_examples",
    "state_street_bleu_rouge",
    "state_street_bleu_rouge",
    "state_street_bleu_rouge",
    "state_street_bleu_rouge",
    "arrival_delay_regression",
    "airline_delay",
    "fdl_enrichment_test",
    "fdl_enrichment_test2",
    "fiddler_rag_llm_cb_2",
    "fiddler_rag_llm_chatbot",
    "hybrid_bank_churn_model",
    "fiddler_rag_llm",
    "fiddler_rag_llm_chatbot",
    "model_binary_relevance",
    "model_graded_relevance",
    "search_ranking_example",
    "model_graded_relevance_ranking",
    "model_graded_relevance_ranking",
    "resnet18",
    "search_ranking_example",
    "search_ranking_example2",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "fiddler_rag_llm_chatbot232",
    "fiddler_rag_llm_chatbot232",
    "fiddler_rag_llm_chatbot232",
    "resnet18",
    "nlp_newsgroups_multiclass",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "search_ranking_trips",
    "search_ranking_trips",
    "trip_assistant_chatbot",
    "assistant_chatbot",
    "search_ranking",
    "assistant_chatbot",
    "search_ranking",
    "bank_churn_simple_monitoring",
    "nlp_newsgroups_multiclass_njn",
    "test_dec3",
    "fiddler_rag_llm_chatbot_b1",
    "bank_churn_simple_monitoring",
    "fiddler_llm_evaluation",
    "fiddler_llm_evaluation1",
    "fiddler_llm_evaluation2",
    "fiddler_llm_evaluation_jab",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot1",
    "fiddler_rag_llm_chatbot_eval",
    "resnet18",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot2",
    "fiddler_rag_llm_chatbot3",
    "bank_churn",
    "resnet18",
    "model_18ec_bin",
    "resnet18",
    "resnet18_2",
    "resnet18",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot",
    "ust_fiddler_rag_llm",
    "clothing_classifier",
    "fiddler_rag_llm_chatbot_meee",
    "resnet18",
    "yolo11",
    "yolov5_embed_monitoring",
    "yolov5_embed_test",
    "yolov5_embed_test2",
    "yolo_v5su_binary",
    "yolo_v5su_multi",
    "yolo_v5su_test",
    "yolov5_xai",
    "seabed_classifier_xai_max",
    "seabed_max_embeddings",
    "nick_nba_notset",
    "bank_churn",
    "bank_churn",
    "bank_churn",
    "bank_churn",
    "nidlk_0",
    "bank_churn_simple",
    "churn_classifier",
    "fiddler_rag_chatbot",
    "model_binary_relevance",
    "model_graded_relevance",
    "arrival_delay_regression",
    "bank_churn",
    "xai_bank_churn",
    "clothing_classifier",
    "imdb_rnn",
    "yolov5",
    "logreg_all",
    "christianity_atheism",
    "fiddler_rag_llm_chatbot_me",
    "fiddler_rag_llm_chatbot_b",
    "bank_churn_simple_monitoring",
    "fiddler_llm_evaluation",
    "fiddler_rag_llm_chatbot",
    "imbalance_cc_fraud",
    "imbalance_cc_fraud_weighted",
    "bank_churn_d",
    "bank_churn_simple_monitoring",
    "bank_churn_simple_dg",
    "search_ranking_graded_rel",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot",
    "model_abc",
    "readmission_multiclassifier",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot2",
    "logreg_all",
    "hybrid_bank_churn_model",
    "hybrid_lending_model",
    "fraud_detection_model_v1",
    "fraud_detection_model_v2",
    "imbalance_cc_fraud",
    "imbalance_cc_fraud_weighted",
    "iris_classification",
    "newsgroups_openai",
    "newsgroups_tfidf",
    "readmission_multiclassifier",
    "intersectionally_unfair",
    "bank_churn_simple_monitoring",
    "bank_churn_1",
    "bank_churn_1_wertyu",
    "test_draft_model",
    "churn_classifier",
    "search_ranking_update",
    "search_ranking_update",
    "hybrid_bank_churn_model",
    "hybrid_bank_churn_model",
    "bank_churn_model_versions",
    "bank_churn_model_versions",
    "bank_churn_model_versions",
    "bank_churn_model_versions",
    "churn_classifier",
    "fiddler_llm_evaluation",
    "xyzabc",
    "fiddler_rag_llm_chatbot",
    "fiddler_rag_llm_chatbot_l1",
    "bank_churn_simple_monitoring",
]

In [54]:
len(raw_res)

228